# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
if 'uk_food' in mongo.list_database_names():
    print("The 'uk_food' database exists.")
else:
    print("The 'uk_food' database does not exist.")

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
print(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 New Restaurant Street",
    "AddressLine2": "Greenwich",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 XXX",
    "Phone": "123-456-7890",
    "RatingValue": "",
    "RatingKey": "",
    "RatingDate": "",
    "LocalAuthorityCode": "123",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "",
    "LocalAuthorityEmailAddress": "",
    "scores": {
        "Hygiene": 0,
        "Structural": 0,
        "ConfidenceInManagement": 0
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "",
        "latitude": ""
    },
    "RightToReply": "",
    "Distance": 0,
    "NewRatingPending": True,
    "meta": {
        "dataSource": None,
        "extractDate": "0001-01-01T00:00:00",
        "itemCount": 0,
        "returncode": None,
        "totalCount": 0,
        "totalPages": 0,
        "pageSize": 0,
        "pageNumber": 0
    },
    "links": [
        {
            "rel": "self",
            "href": ""
        }
    ]
}


In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant_data)


In [ ]:
# Check that the new restaurant was inserted
results = establishments.find({'BusinessName':'Penang Flavours'})
for result in results:
    print(result)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

# Projection to return only the specified fields (BusinessTypeID and BusinessType)
projection = {
    "_id": 0,
    "BusinessTypeID": 1,
    "BusinessType": 1
}

# Execute the query and projection
result = establishments.find_one(query, projection)

# Print the result
print(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {
    "BusinessName": "Penang Flavours"
}

# Define the update operation to set the correct BusinessTypeID
update_operation = {
    "$set": {
        "BusinessTypeID": 1  # Assign the correct BusinessTypeID here
    }
}

# Update the new restaurant document in the database
update_result = establishments.update_one(filter_query, update_operation)

In [ ]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("New restaurant 'Penang Flavours' updated successfully.")
else:
    print("Failed to update the new restaurant.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Deleted {delete_result.deleted_count} documents in Dover Local Authority")

In [ ]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover' after deletion: {remaining_count}")

In [ ]:
# Check that other documents remain with 'find_one'
example_document = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
example_document

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

In [ ]:
# Check that the coordinates and rating value are now numbers
# Find a document with latitude and longitude
document_with_coordinates = establishments.find_one({"geocode.latitude": {"$type": "double"}, "geocode.longitude": {"$type": "double"}})
print("Coordinates (Latitude and Longitude):")
print(document_with_coordinates["geocode"]["latitude"], document_with_coordinates["geocode"]["longitude"])
# Find a document with a numeric RatingValue
document_with_numeric_rating = establishments.find_one({"RatingValue": {"$type": "number"}})
print("Rating Value:")
print(document_with_numeric_rating["RatingValue"])

